<a href="https://colab.research.google.com/github/dcovarru/Python/blob/master/Covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#####################
# Import packages
#####################

In [0]:
#####################
# Import NYT data 
#####################

In [0]:
#####################
# ETL data 
#####################

In [0]:
#####################
# Create Model with metrics?
#####################

In [0]:
#####################
# Trend Diagnostics
#####################